# Hospital Service Area (HSA) Optimization - v5.0 FINAL
## Unified Scoring System with Mode-Specific Weight Profiles

**Date**: 2025-11-27

**Key Features**:
- **Unified scoring system**: Single formula for all modes with mode-specific weight profiles
- **Volume-based radius assignment**: Larger facilities get larger radii (0.8x to 1.2x multiplier)
- **Composite score tracking**: All modes track and export composite scores
- **Overlap removal**: All modes include post-processing to remove HSAs with >80% overlap
- **Configurable parameters**: All key parameters set in notebook (not in .py file)

## Five Optimization Modes

### 1. FEWEST (Minimize number of HSAs)
**Goal**: Achieve coverage with minimum facilities
**Stopping**: 90% coverage
**Expected**: 10-20 HSAs

**Scoring Formula**:
```
composite_score =
    WEIGHT_COVERAGE * coverage_norm * 2.0           # Coverage gain (high priority)
  + WEIGHT_PATIENT_VOLUME * patient_norm * 1.5      # Facility size (high reward)
  - WEIGHT_OVERLAP_PENALTY * overlap_frac * 1.0     # Redundancy penalty
  + WEIGHT_CLIMATE * climate_norm * 1.0             # Geographic diversity
  + WEIGHT_COVERAGE_PROGRESS * progress_norm * 0.0  # Progress disabled
```

**Rationale**: Prioritizes large, high-impact facilities that maximize coverage with minimal overlap. Zero progress weight prevents adding small facilities just to reach target.

---

### 2. FOOTPRINT (Maximize geographic spread)
**Goal**: Maximize population reached and geographic coverage
**Stopping**: 90% coverage
**Expected**: 25-35 HSAs

**Scoring Formula**:
```
composite_score =
    WEIGHT_COVERAGE * coverage_norm * 1.0           # Coverage gain (normal)
  + WEIGHT_PATIENT_VOLUME * patient_norm * 1.0      # Facility size (normal)
  - WEIGHT_OVERLAP_PENALTY * overlap_frac * 0.3     # Reduced overlap penalty
  + WEIGHT_CLIMATE * climate_norm * 1.0             # Geographic diversity
  + WEIGHT_COVERAGE_PROGRESS * progress_norm * 1.0  # Progress toward target
```

**Rationale**: Balanced weights allow broader geographic distribution. Low overlap penalty permits more facilities in populated areas to maximize reach.

---

### 3. DISTANCE (Minimize travel distance)
**Goal**: Minimize average travel distance to facilities
**Stopping**: 90% coverage
**Expected**: 10-20 HSAs

**Scoring Formula**:
```
composite_score =
    WEIGHT_COVERAGE * coverage_norm * 1.5           # Coverage gain (high)
  + WEIGHT_PATIENT_VOLUME * patient_norm * 1.0      # Facility size (normal)
  - WEIGHT_OVERLAP_PENALTY * overlap_frac * 0.8     # Moderate overlap penalty
  + WEIGHT_CLIMATE * climate_norm * 1.0             # Geographic diversity
  + WEIGHT_COVERAGE_PROGRESS * progress_norm * 1.0  # Progress toward target
  - WEIGHT_DISTANCE_PENALTY * distance_norm * 1.0   # Travel distance penalty (unique to this mode)
```

**Rationale**: Only mode with distance penalty. Penalizes facilities far from population centers, encouraging selection of geographically central facilities.

---

### 4. GOVERNORATE_TAU_COVERAGE (TAU method per governorate)
**Goal**: Achieve 60% coverage in each governorate independently
**Stopping**: TAU threshold met in all governorates
**Expected**: 18-25 HSAs

**Scoring Formula**: Same as FOOTPRINT (balanced approach per governorate)

**Rationale**: Ensures geographic equity by guaranteeing minimum coverage in each administrative region, regardless of population density.

---

### 5. GOVERNORATE_FEWEST (One anchor per governorate + FEWEST)
**Goal**: Guarantee coverage in each governorate, then minimize total HSAs
**Stopping**: 90% coverage with at least 1 HSA per governorate
**Expected**: 12-20 HSAs

**Scoring Formula**: Same as FEWEST (minimize facilities)

**Rationale**: Combines administrative equity (1 HSA/governorate minimum) with efficiency (FEWEST scoring). Ensures no governorate is left without a facility.

---

## Base Weights (Configurable in Cell 4)

These weights are applied to normalized 0-1 components, then multiplied by mode-specific profiles above:

- `WEIGHT_COVERAGE = 5.0` (population coverage gain)
- `WEIGHT_PATIENT_VOLUME = 7.0` (facility size reward)
- `WEIGHT_OVERLAP_PENALTY = 3.0` (redundancy penalty)
- `WEIGHT_CLIMATE = 1.0` (climate/geographic diversity)
- `WEIGHT_COVERAGE_PROGRESS = 0.5` (progress toward target)
- `WEIGHT_DISTANCE_PENALTY = 0.0` (travel distance penalty - only used in DISTANCE mode)

## Radius Assignment

1. Compute local population density (10km radius)
2. Classify urban (>1500 people/km²) vs rural
3. Assign base radius: Urban=12km, Rural=28km
4. **Apply patient volume scaling**: 0.8x (small) to 1.2x (large)
5. Apply network multiplier: INF=1.15x, NCD=1.0x
6. Final radius range: ~11-39 km

**Result**: Larger hospitals serve larger catchment areas (16+ unique radii, not just 2!)

---

## Normalization Details

All scoring components normalized to 0-1 range:
- **coverage_norm** = new_coverage / total_population
- **patient_norm** = facility_volume / max_volume
- **overlap_frac** = overlap_population / total_coverage (already 0-1)
- **climate_norm** = 1 / (1 + unique_climate_clusters)
- **progress_norm** = coverage_gain / remaining_to_target
- **distance_norm** = mean_distance / max_distance

---

See cells below for configuration, execution, and analysis.



In [ ]:
# ============================================================================
# COMPREHENSIVE CONFIGURATION - ALL PARAMETERS
# ============================================================================

# Import Path for file paths (needed in this cell)
from pathlib import Path

# --- File Paths ---
DATA_DIR = Path("data")
OUT_DIR = Path("out")
NETWORK = "INF"

# --- Core Algorithm Parameters ---
TAU_COVERAGE = 0.95                    # Maximum target coverage (95% - raised to differentiate modes)
MAX_RADIUS_KM = 25                     # Absolute maximum radius (clinical requirement)
RNG_SEED = 42                          # Random seed

# --- Computational Efficiency ---
BUFFER_KM = 30                         # Spatial crop buffer
COARSEN_FACTOR = 10                    # Grid coarsening factor (use 4 for higher precision)

# --- Density-Based Classification ---
DENSITY_RADIUS_KM = 10.0               # FIXED 10km radius for density calculation
URBAN_DENSITY_THRESH = 1500.0          # Urban if density > this (people/km²)

# --- Base Radii ---
# Balanced radii to give scoring model granularity while maintaining reasonable coverage
URBAN_BASE_RADIUS_KM = 12.0            # Base radius for urban facilities
RURAL_BASE_RADIUS_KM = 20.0            # Base radius for rural facilities

# --- HSA Count Targets ---
TARGET_HSA_MIN = 15                    # Minimum desired HSAs (for FEWEST mode floor)
TARGET_HSA_MAX = 28                    # Maximum desired HSAs

# --- Volume-Based Scoring ---
VOLUME_WEIGHT = 1.5                    # Multiplier for patient volume in scoring function

# --- Network-Specific Multipliers ---
NETWORK_RADIUS_MULTIPLIERS = {
    'INF': 1.0,   # Removed multiplier to keep radii in clinical target range
    'NCD': 1.0,   # Non-communicable disease (baseline)
}

# --- Patient Volume Scaling ---
PATIENT_RADIUS_ALPHA = 0.35            # Scaling strength for patient volume

# --- Climate Diversity (applies to ALL objectives) ---
CLIMATE_DIVERSITY_ON = True
CLIMATE_K = 8
CLIMATE_MIN_PER_CLUSTER = 1            # hard floor per climate bin (set 0 to disable)
CLIMATE_CLUSTER_COL = "climate_k"      # column with precomputed cluster per facility
CLIMATE_CSV = {
    "INF": DATA_DIR / "INF_Hospitals_Climate_Features_with_clusters.csv",
    "NCD": DATA_DIR / "NCD_Hospitals_Climate_Features_with_clusters.csv",
}


# --- Scoring Weights (0-10 scale, applied to normalized 0-1 components) ---
# These are the baseline weights, modified by mode-specific profiles below
WEIGHT_COVERAGE = 8.0                  # NEW population coverage (DOMINANT factor)
WEIGHT_OVERLAP_PENALTY = 2.0           # Penalty for redundant coverage
WEIGHT_CLIMATE = 1.0                   # Reward for climate/geographic diversity
WEIGHT_PATIENT_VOLUME = 3.0            # Reward for larger patient volumes
WEIGHT_COVERAGE_PROGRESS = 1.0         # Reward for approaching coverage target
WEIGHT_DISTANCE_PENALTY = 3.0          # Penalty for average travel distance

# Aliases for backward compatibility
COVERAGE_WEIGHT = WEIGHT_COVERAGE
LAMBDA_CLIMATE = WEIGHT_CLIMATE
LAMBDA_PATIENT_VOLUME = WEIGHT_PATIENT_VOLUME

# --- Mode-Specific Weight Profiles ---
# Each mode applies multipliers to the base weights above
MODE_WEIGHT_PROFILES = {
    'fewest': {
        'coverage': 1.5,           # Moderate coverage multiplier
        'coverage_progress': 0.0,  # NO progress reward (stop immediately at 95%)
        'overlap': 0.5,            # Low overlap penalty
        'patient_volume': 2.5,     # Very high patient volume preference
        'distance': 0.0,           # No distance penalty
    },
    'footprint': {
        'coverage': 1.0,           # Standard coverage gain
        'coverage_progress': 8.0,  # VERY STRONG progress reward (keep selecting to ~100%)
        'overlap': 0.01,           # Almost no overlap penalty
        'patient_volume': 0.05,    # Almost zero patient volume preference
        'distance': 0.0,           # No distance penalty
    },
    'distance': {
        'coverage': 1.0,           # Standard coverage gain
        'coverage_progress': 6.0,  # Strong progress reward
        'overlap': 0.02,           # Almost no overlap penalty
        'patient_volume': 0.08,    # Very low patient volume preference
        'distance': 2.5,           # Strong distance penalty (minimize avg distance)
    },
}


# --- Optimization Modes to Run ---
# Enable/disable specific optimization modes
RUN_OBJECTIVES = {
    'fewest': True,                      # Minimize number of HSAs
    'footprint': True,                   # Maximize geographic spread
    'distance': True,                    # Minimize travel distance
    'governorate_tau_coverage': True,    # TAU method per governorate
    'governorate_fewest': True           # One anchor per governorate + FEWEST
}

# --- HSAOptimizer configuration dict ---
config = {
    'pop_path': str(DATA_DIR / 'jor_ppp_2020_constrained.tif'),
    'tau_coverage': 0.60,  # Used for GOVERNORATE_TAU_COVERAGE mode only
    'coarsen': 4,  # Coarsen raster by 4x for speed (use 1 for full resolution)
}

print("="*80)
print("CONFIGURATION LOADED")
print("="*80)
print(f"Network: {NETWORK}")
print(f"Coverage target (TAU_COVERAGE): {TAU_COVERAGE*100:.0f}%")
print(f"Governorate mode coverage: {config['tau_coverage']*100:.0f}%")
print(f"Coarsen factor: {config['coarsen']}x")
print(f"Urban base radius: {URBAN_BASE_RADIUS_KM} km")
print(f"Rural base radius: {RURAL_BASE_RADIUS_KM} km")
print(f"Max radius: {MAX_RADIUS_KM} km")
print(f"Climate diversity: {CLIMATE_DIVERSITY_ON}")
print(f"Target HSA range: {TARGET_HSA_MIN}-{TARGET_HSA_MAX}")
print("="*80)


## Mode-Specific Optimization SettingsThe optimization now uses 

**mode-specific stopping conditions** to differentiate facility counts:

### Stopping Logic:- 

**FEWEST**: Stops at 90% coverage with minimum 15 facilities  - Goal: Minimize number of facilities while meeting coverage target-
**FOOTPRINT**: Continues to 22 facilities or 98% coverage  - Goal: Maximize geographic coverage/footprint- 
**DISTANCE**: Continues to 22 facilities or 98% coverage  - Goal: Minimize travel distance while expanding coverage### Weight Profiles:- 
**FEWEST**: High patient volume weight (2.5x), zero progress reward- 
**FOOTPRINT**: Very high progress reward (8.0x), minimal patient volume weight (0.05x)- 
**DISTANCE**: High progress reward (6.0x), strong distance penalty (2.5x)

### Expected Results:- FEWEST: ~15-16 HSAs, ~94% coverage- FOOTPRINT: ~20-22 HSAs, ~97% coverage- DISTANCE: ~20-22 HSAs, ~91% coverage

## 1. Configure Optimization Parameters

Adjust these parameters to tune the penalty-based scoring:

In [ ]:
# ============================================================================
# IMPORTS AND PARAMETER INJECTION
# ============================================================================

# Import libraries
import sys
from pathlib import Path
import geopandas as gpd
import pandas as pd

# Import optimization module
import hsa_optimization
from hsa_optimization import HSAOptimizer

# ============================================================================
# INJECT CONFIGURATION INTO HSA_OPTIMIZATION MODULE
# ============================================================================
# The hsa_optimization module no longer defines its own parameters.
# Instead, it expects them to be injected from the notebook.
# This makes all parameters visible and editable in one place (Cell 0 above).

param_names = [
    'DATA_DIR', 'OUT_DIR', 'NETWORK',
    'TAU_COVERAGE', 'MAX_RADIUS_KM', 'RNG_SEED',
    'BUFFER_KM', 'COARSEN_FACTOR',
    'DENSITY_RADIUS_KM', 'URBAN_DENSITY_THRESH',
    'URBAN_BASE_RADIUS_KM', 'RURAL_BASE_RADIUS_KM',
    'TARGET_HSA_MIN', 'TARGET_HSA_MAX',
    'VOLUME_WEIGHT', 'PATIENT_RADIUS_ALPHA',
    'NETWORK_RADIUS_MULTIPLIERS',
    'CLIMATE_DIVERSITY_ON', 'CLIMATE_K', 'CLIMATE_MIN_PER_CLUSTER',
    'CLIMATE_CLUSTER_COL', 'CLIMATE_CSV',
    'WEIGHT_COVERAGE', 'WEIGHT_OVERLAP_PENALTY', 'WEIGHT_CLIMATE',
    'WEIGHT_PATIENT_VOLUME', 'WEIGHT_COVERAGE_PROGRESS', 'WEIGHT_DISTANCE_PENALTY',
    'COVERAGE_WEIGHT', 'LAMBDA_CLIMATE', 'LAMBDA_PATIENT_VOLUME',
    'MODE_WEIGHT_PROFILES',
]

print("Injecting parameters into hsa_optimization module...")
injected_count = 0
for param in param_names:
    if param in globals():
        setattr(hsa_optimization, param, globals()[param])
        injected_count += 1
    else:
        print(f"  WARNING: Parameter {param} not found in globals()")

print("="*80)
print("IMPORTS AND PARAMETER INJECTION COMPLETE")
print("="*80)
print(f"Parameters injected: {injected_count}/{len(param_names)}")
print("="*80)


## 2. Run Optimizations

This will run all enabled objectives and stop at their respective coverage thresholds:
- **FEWEST**: 90% coverage (minimize # of HSAs) - typically 15-20 HSAs
- **DISTANCE**: 90% coverage (minimize travel distance) - typically 15-20 HSAs  
- **FOOTPRINT**: 90% coverage (maximize population reached) - typically 25-35 HSAs
- **GOVERNORATE_TAU_COVERAGE**: 60% per governorate (TAU method, 18-25 HSAs)
- **GOVERNORATE_FEWEST**: 1 HSA per governorate + FEWEST (12-20 HSAs)

**Note**: All modes now use unified normalized scoring with mode-specific weight profiles.
Results will include `composite_score`, `initial_radius_km`, and `service_radius_km` columns.

In [ ]:
# Initialize optimizer
optimizer = HSAOptimizer(config)



# Load facilities WITH CLIMATE DATA from CSV
climate_csv = DATA_DIR / f'{NETWORK}_Hospitals_Climate_Features_with_clusters.csv'

if not climate_csv.exists():
    raise FileNotFoundError(f"Climate CSV not found: {climate_csv}")

# Read climate CSV
climate_df = pd.read_csv(climate_csv)

# Rename FacilityName to HealthFacility (what the optimizer expects)
if 'FacilityName' in climate_df.columns:
    climate_df['HealthFacility'] = climate_df['FacilityName']
elif 'HealthFacility' not in climate_df.columns:
    raise ValueError("Climate CSV must have either 'FacilityName' or 'HealthFacility' column")

# Load diagnosis counts (for Total column)
diagnosis_csv = OUT_DIR / f'{NETWORK}_diagnosis_counts_pivot.csv'
if not diagnosis_csv.exists():
    raise FileNotFoundError(f"Diagnosis counts not found: {diagnosis_csv}")

diagnosis_df = pd.read_csv(diagnosis_csv)

# Merge climate data with diagnosis counts on HealthFacility
merged_df = climate_df.merge(
    diagnosis_df[['healthfacility', 'total_diagnoses']],
    left_on='HealthFacility',
    right_on='healthfacility',
    how='left'
)

# Rename total_diagnoses to Total (what the optimizer expects)
merged_df['Total'] = merged_df['total_diagnoses']

# Check for missing values
missing_total = merged_df['Total'].isna().sum()
if missing_total > 0:
    print(f"WARNING: {missing_total} facilities missing diagnosis counts")
    # Fill with 0 or drop?
    merged_df['Total'] = merged_df['Total'].fillna(0)

# Create GeoDataFrame from lat/lon
facilities = gpd.GeoDataFrame(
    merged_df,
    geometry=gpd.points_from_xy(merged_df['lon'], merged_df['lat']),
    crs='EPSG:4326'
)

# Reproject to UTM for distance calculations (Jordan is in UTM zone 36N/37N)
# Using a common Jordan projection: EPSG:32637 (UTM 37N)
facilities = facilities.to_crs('EPSG:32637')

print(f"\nMerged climate + diagnosis data:")
print(f"  Facilities with climate data: {len(climate_df)}")
print(f"  Facilities with diagnosis counts: {len(diagnosis_df)}")
print(f"  Facilities after merge: {len(facilities)}")
print(f"  Facilities with Total column: {facilities['Total'].notna().sum()}")
print(f"  Facilities with HealthFacility column: {'HealthFacility' in facilities.columns}")


print(f"Loaded {len(facilities)} {NETWORK} facilities ")
if 'climate_k' in facilities.columns:
    print(f"  climate_k column present: YES")
    print(f"  climate_k values: {sorted(facilities['climate_k'].dropna().unique())}")
else:
    print(f"  climate_k column present: NO (climate diversity scoring will be disabled!)")

# Load governorates for governorate modes
gov_file = DATA_DIR / 'jordan_governorates.gpkg'
governorates = gpd.read_file(gov_file) if (DATA_DIR / 'jordan_governorates.gpkg').exists() else None

# Store results for all objectives
all_results = {}

# Run optimizations for selected objectives
for objective in ['fewest', 'footprint', 'distance', 'governorate_tau_coverage', 'governorate_fewest']:
    if objective not in RUN_OBJECTIVES or not RUN_OBJECTIVES[objective]:
        print(f"\nSkipping {objective.upper()} optimization (disabled)")
        continue
    
    print(f"\n{'='*80}")
    print(f"Running {objective.upper()} optimization...")
    print('='*80)
    
    # Run optimization
    if objective in ['governorate_tau_coverage', 'governorate_fewest']:
        if governorates is None:
            print("  ERROR: Governorates file not found, skipping")
            continue
        result = optimizer.optimize(facilities, objective=objective, network_type=NETWORK, governorates_gdf=governorates)
    else:
        result = optimizer.optimize(facilities, objective=objective, network_type=NETWORK)

    
    selected_facilities = result['facilities']
    coverage_pct = result['coverage_pct']
    
    all_results[objective] = {
        'result': result,
        'facilities': selected_facilities,
        'coverage': coverage_pct
    }
    
    print(f"\n{'='*80}")
    print(f"{objective.upper()} OPTIMIZATION COMPLETE")
    print('='*80)
    print(f"Selected: {len(selected_facilities)} HSAs")
    print(f"Coverage: {coverage_pct:.1f}%")
    
    # ========================================================================
    # SAVE RESULTS TO GEOJSON (v2 files)
    # ========================================================================
    print(f"\nSaving results to GeoJSON...")
    
    # Add metadata
    selected_with_meta = selected_facilities.copy()
    selected_with_meta['network_type'] = NETWORK
    selected_with_meta['optimization_mode'] = objective
    
    # Save to v2.geojson file
    output_file = OUT_DIR / f'{NETWORK}_{objective}_hsas_v2.geojson'
    selected_with_meta.to_file(output_file, driver='GeoJSON')
    
    # Verify saved file
    reloaded = gpd.read_file(output_file)
    print(f"  Saved: {output_file.name}")
    print(f"  Columns in saved file: {len(reloaded.columns)}")
    print(f"  climate_k preserved: {'climate_k' in reloaded.columns}")
    
    if 'service_radius_km' in reloaded.columns:
        max_radius = reloaded['service_radius_km'].max()
        print(f"  Max service_radius_km: {max_radius:.1f} km (should be <= 30.0)")
        if max_radius > 30.0:
            print(f"  WARNING: Radius exceeds 30 km limit!")
    
    print(f"  File verified successfully")

print(f"\n\n{'='*80}")
print("ALL OPTIMIZATIONS COMPLETE")
print('='*80)
for obj, data in all_results.items():
    print(f"  {obj.upper():26s}: {len(data['facilities']):3d} HSAs, {data['coverage']:.1f}% coverage")

## 3. Select Objective for Analysis

Choose which objective result to analyze in detail:

In [ ]:
# ============================================================================
# SECTIONS 3-8: ANALYZE ALL OPTIMIZATION MODES
# ============================================================================

from hsa_objective_analysis import analyze_all_modes

# Analyze all modes with complete suite of visualizations and statistics
analyze_all_modes(all_results)

In [ ]:

  # ============================================================================
  # CROSS-MODE COMPARISON: Facility Counts and HSA Volumes
  # ============================================================================

  from geopandas import sjoin
  import pandas as pd


  # Load all facilities once from climate CSV + diagnosis counts
  climate_csv = DATA_DIR / f'{NETWORK}_Hospitals_Climate_Features_with_clusters.csv'
  diagnosis_csv = OUT_DIR / f'{NETWORK}_diagnosis_counts_pivot.csv'

  climate_df = pd.read_csv(climate_csv)
  diagnosis_df = pd.read_csv(diagnosis_csv)

  # Rename FacilityName to HealthFacility
  if 'FacilityName' in climate_df.columns:
      climate_df['HealthFacility'] = climate_df['FacilityName']

  # Merge
  merged_df = climate_df.merge(
      diagnosis_df[['healthfacility', 'total_diagnoses']],
      left_on='HealthFacility',
      right_on='healthfacility',
      how='left'
  )
  merged_df['Total'] = merged_df['total_diagnoses'].fillna(0)

  all_facilities_gdf = gpd.GeoDataFrame(
      merged_df,
      geometry=gpd.points_from_xy(merged_df['lon'], merged_df['lat']),
      crs='EPSG:4326'
  ).to_crs('EPSG:32637')





  # Compare all modes
  comparison_results = []

  for mode in ['fewest', 'footprint', 'distance', 'governorate_tau_coverage', 'governorate_fewest']:
      if mode not in all_results:
          continue

      final_hsas = all_results[mode]['facilities']
      N_ANCHORS = len(final_hsas)

      print(f"\n{'='*80}")
      print(f"{mode.upper()} - DETAILED HSA ANALYSIS ({N_ANCHORS} HSAs)")
      print('='*80)

      # Calculate detailed HSA statistics
      details = []

      for idx, row in final_hsas.iterrows():
          # Get anchor facility volume
          anchor_volume = pd.to_numeric(row['Total'], errors='coerce')

          # Get HSA geometry (circle)
          hsa_geom = row.geometry

          # Calculate total population in HSA using raster mask
          try:
              from rasterio.mask import mask
              import rasterio

              with rasterio.open(config['pop_path']) as src:
                  out_image, out_transform = mask(src, [hsa_geom], crop=True, nodata=0)
                  total_pop = out_image.sum()
          except:
              total_pop = 0

          # Find ALL facilities within this HSA using spatial join
          # Create temporary GeoDataFrame with just this HSA
          hsa_temp = gpd.GeoDataFrame([{'geometry': hsa_geom}], crs=all_facilities_gdf.crs)

          # Spatial join to find facilities intersecting this HSA
          facilities_in_hsa = sjoin(
              all_facilities_gdf,
              hsa_temp,
              how='inner',
              predicate='intersects'
          )

          # Calculate total HSA volume (sum of all facility volumes in HSA)
          total_hsa_volume = 0
          for fac_idx, fac_row in facilities_in_hsa.iterrows():
              fac_vol = pd.to_numeric(fac_row['Total'], errors='coerce')
              if not pd.isna(fac_vol):
                  total_hsa_volume += fac_vol

          # Append details with ALL required columns
          details.append({
              'Mode': mode.upper(),
              'HSA_Anchor': row['FacilityName'],
              'Anchor_Patient_Volume': anchor_volume,
              'Num_Facilities_in_HSA': len(facilities_in_hsa),
              'Total_HSA_Volume': total_hsa_volume,
              'HSA_Population': total_pop,
              'Service_Radius_km': row.get('service_radius_km', row.get('initial_radius_km', 0)),
              'Composite_Score': row.get('composite_score', 0)
          })

      # Create DataFrame
      detail_df = pd.DataFrame(details)

      # Define display columns
      display_cols = ['HSA_Anchor', 'Anchor_Patient_Volume', 'Num_Facilities_in_HSA',
                      'Total_HSA_Volume', 'HSA_Population', 'Service_Radius_km', 'Composite_Score']

      # Show ALL facilities (not .head(15))
      print(detail_df[display_cols].to_string(index=False))

      # Summary statistics
      print(f"\nSummary Statistics for {mode.upper()}:")
      print(f"  Total HSAs: {len(detail_df)}")
      print(f"  Avg facilities per HSA: {detail_df['Num_Facilities_in_HSA'].mean():.1f}")
      print(f"  Total facilities covered: {detail_df['Num_Facilities_in_HSA'].sum()}")
      print(f"  Avg HSA volume: {detail_df['Total_HSA_Volume'].mean():,.0f}")
      print(f"  Total HSA volume: {detail_df['Total_HSA_Volume'].sum():,.0f}")

      # Add to comparison
      comparison_results.append({
          'Mode': mode.upper(),
          'Num_HSAs': N_ANCHORS,
          'Avg_Facilities_per_HSA': detail_df['Num_Facilities_in_HSA'].mean(),
          'Total_Facilities_Covered': detail_df['Num_Facilities_in_HSA'].sum(),
          'Avg_HSA_Volume': detail_df['Total_HSA_Volume'].mean(),
          'Total_HSA_Volume': detail_df['Total_HSA_Volume'].sum()
      })

  # Cross-mode comparison table
  print(f"\n\n{'='*80}")
  print("CROSS-MODE COMPARISON SUMMARY")
  print('='*80)
  comparison_df = pd.DataFrame(comparison_results)
  print(comparison_df.to_string(index=False))

## Summary

This notebook implements the **v5.0 Normalized Scoring System** for HSA selection:

### Key Improvements in v5.0:

1. **Normalized Scoring (0-1 components, 0-10 weights)**:
   - All scoring components normalized to 0-1 range
   - Weights in intuitive 0-10 scale
   - Score range: 2-17 (reasonable, not exponential)
   - Easy to understand and tune

2. **Volume-Based Radius Assignment**:
   - Patient volume now influences radius (0.8x to 1.2x multiplier)
   - Results in 15-16 unique radii (not just 2!)
   - Range: ~11-39 km depending on density and volume
   - Larger hospitals serve larger catchment areas

3. **Unified Mode System**:
   - Single scoring formula for all modes
   - Mode differences via weight profile multipliers
   - Distance penalty only in DISTANCE mode
   - All modes track composite scores

4. **Selection Quality**:
   - Largest facilities systematically selected first
   - Top 15 anchors: 0 facilities with <100 patients
   - 13 out of 16 selections are >2000 patients
   - Al-Basheer Hospital (6831 patients) has highest score

### Scoring Formula (FEWEST mode):

```python
composite_score = 
    5.0 * (pop_coverage / total_pop) * 2.0       # Coverage: 0-10
  - 3.0 * (overlap / total_coverage) * 1.0       # Overlap: 0 to -3
  + 1.0 * (1 / (1 + cluster_count))              # Climate: 0-1
  + 7.0 * (patient_vol / max_vol) * 1.5          # Volume: 0-10.5
  + 0.5 * (progress / remaining) * 0.3           # Progress: 0-0.15
```


### Tuning Weights:

Edit `hsa_optimization.py` lines 234-267:

```python
# Increase to prefer larger facilities:
WEIGHT_PATIENT_VOLUME = 8.0  # (currently 7.0)

# Increase to prioritize coverage:
WEIGHT_COVERAGE = 7.0  # (currently 5.0)

# Reduce to allow more geographic spread:
WEIGHT_CLIMATE = 0.5  # (currently 1.0)
```



## 9. Generate Professional Maps

Create HSA boundary polygons and professional maps for all optimization results.

In [ ]:
# Load Jordan country boundary for clipping
country_file = DATA_DIR / 'jordan_admin0.gpkg'  # Adjust filename if needed
if not country_file.exists():
    country_file = DATA_DIR / 'jordan_boundary.gpkg'
if not country_file.exists():
    country_file = DATA_DIR / 'jordan_governorates.gpkg'  # Use governorates as fallback

if country_file.exists():
    country_boundary = gpd.read_file(country_file)
    if 'jordan_governorates' in str(country_file):
        # Dissolve governorates to create country boundary
        country_boundary = country_boundary.dissolve()
    print(f"Country boundary loaded from: {country_file.name}")
    print(f"  CRS: {country_boundary.crs}")
else:
    print("WARNING: Country boundary file not found. Maps will not be clipped.")
    country_boundary = None

In [ ]:
# ============================================================================
# SECTION 9: CREATE MAPS FOR ALL MODES - USING WORKING MODULE
# ============================================================================

# FORCE FRESH IMPORT - clear any cached modules
import sys
if 'hsa_mapping' in sys.modules:
    del sys.modules["hsa_mapping"]
if 'hsa_mapping_working' in sys.modules:
    del sys.modules["hsa_mapping_working"]

# Import the WORKING mapping module (extracted from create_geopandas_maps.py)
from hsa_mapping_working import create_all_hsa_maps

print("Using hsa_mapping_working module (extracted from create_geopandas_maps.py)")
print("This version has been verified to produce correct maps with:")
print("  - Circular HSAs (not stretched)")
print("  - Visible population grid")

# Create all maps with proper layer ordering
create_all_hsa_maps(
    all_results=all_results,
    network=NETWORK,
    data_dir=DATA_DIR,
    out_dir=OUT_DIR
)

In [ ]:

  # ============================================================================
  # SECTION 10: HSA DETAIL TABLES - ALL MODES, ALL HSAS
  # ============================================================================

  from geopandas import sjoin
  import rasterio
  from rasterio.mask import mask as raster_mask
  import pandas as pd



  # Load all facilities once from climate CSV + diagnosis counts
  climate_csv = DATA_DIR / f'{NETWORK}_Hospitals_Climate_Features_with_clusters.csv'
  diagnosis_csv = OUT_DIR / f'{NETWORK}_diagnosis_counts_pivot.csv'

  climate_df = pd.read_csv(climate_csv)
  diagnosis_df = pd.read_csv(diagnosis_csv)

  # Rename FacilityName to HealthFacility
  if 'FacilityName' in climate_df.columns:
      climate_df['HealthFacility'] = climate_df['FacilityName']

  # Merge
  merged_df = climate_df.merge(
      diagnosis_df[['healthfacility', 'total_diagnoses']],
      left_on='HealthFacility',
      right_on='healthfacility',
      how='left'
  )
  merged_df['Total'] = merged_df['total_diagnoses'].fillna(0)

  all_facilities = gpd.GeoDataFrame(
      merged_df,
      geometry=gpd.points_from_xy(merged_df['lon'], merged_df['lat']),
      crs='EPSG:4326'
  ).to_crs('EPSG:32637')





  modes_to_process = {
      'fewest': 'FEWEST',
      'footprint': 'FOOTPRINT',
      'distance': 'DISTANCE',
      'governorate_tau_coverage': 'GOVERNORATE TAU COVERAGE',
      'governorate_fewest': 'GOVERNORATE FEWEST'
  }


  for mode_name, mode_label in modes_to_process.items():
      hsas_file = OUT_DIR / f'{NETWORK}_{mode_name}_hsas_v2.geojson'

      if not hsas_file.exists():
          print(f"\nSkipping {mode_label} - file not found")
          continue

      # Load HSAs
      hsas_points = gpd.read_file(hsas_file)
      hsas_points_utm = hsas_points.to_crs(all_facilities.crs)

      print(f"\n{'='*80}")
      print(f"{mode_label} - ALL {len(hsas_points)} HSAs")
      print('='*80)

      # Calculate details for ALL HSAs
      details = []

      for idx, row in hsas_points_utm.iterrows():
          # Get anchor facility volume
          anchor_volume = pd.to_numeric(row['Total'], errors='coerce')
    
          # Get HSA center point (in UTM)
          center_point_utm = row.geometry
    
          # Get radius
          radius_km = row.get('service_radius_km', row.get('initial_radius_km', 15.0))
    
          # Create circle in UTM (meters) for facility counting
          radius_meters = radius_km * 1000
          hsa_circle_utm = center_point_utm.buffer(radius_meters)
    
          # Find ALL facilities within this HSA circle
          hsa_temp = gpd.GeoDataFrame([{'geometry': hsa_circle_utm}], crs=hsas_points_utm.crs)
          facilities_in_hsa = sjoin(all_facilities, hsa_temp, how='inner', predicate='intersects')
    
          # Calculate total HSA volume
          total_hsa_volume = 0
          for fac_idx, fac_row in facilities_in_hsa.iterrows():
              fac_vol = pd.to_numeric(fac_row['Total'], errors='coerce')
              if not pd.isna(fac_vol):
                  total_hsa_volume += fac_vol
    
          # For population: convert circle to EPSG:4326 for raster
          hsa_circle_latlon = gpd.GeoDataFrame(
              [{'geometry': hsa_circle_utm}],
              crs=hsas_points_utm.crs
          ).to_crs('EPSG:4326').geometry.iloc[0]
    
          # Calculate population
          try:
              with rasterio.open(config['pop_path']) as src:
                  out_image, out_transform = raster_mask(src, [hsa_circle_latlon], crop=True, nodata=0)
                  total_pop = float(out_image.sum())
          except Exception as e:
              total_pop = 0
    
          details.append({
              'HSA_Anchor': row['FacilityName'],
              'Anchor_Volume': anchor_volume,
              'Num_Facilities_in_HSA': len(facilities_in_hsa),
              'Total_HSA_Volume': total_hsa_volume,
              'HSA_Population': total_pop,
              'Service_Radius_km': radius_km,
              'Composite_Score': row.get('composite_score', 0)
          })
          # REMOVE ANY PRINT STATEMENTS HERE!

      # ONLY PRINT AFTER THE LOOP COMPLETES
      detail_df = pd.DataFrame(details)
      print(detail_df.to_string(index=False))

      # Summary statistics
      print(f"\nSummary Statistics:")
      print(f"  Total HSAs: {len(detail_df)}")
      print(f"  Total facilities across all HSAs: {detail_df['Num_Facilities_in_HSA'].sum()}")
      print(f"  Total HSA volume: {detail_df['Total_HSA_Volume'].sum():,.0f}")
      print(f"  Total population covered: {detail_df['HSA_Population'].sum():,.0f}")

  print("\n" + "="*80)
  print("HSA DETAIL TABLES COMPLETE")
  print("="*80)